# This code detects QR codes and loads the time and place of detection and local time zone, and information in the QR code in a CSV file.
Video Demo - https://youtu.be/quXhBClXeO0

In [ ]:
#Import libraries
import cv2, time
from pyzbar import pyzbar
from datetime import datetime

## Getting the local time zone

In [ ]:
# importing module
import geocoder 
from timezonefinder import TimezoneFinder
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
# Latitude & Longitude input
Latitude = str(geocoder.ip('me').latlng[0])
Longitude = str(geocoder.ip('me').latlng[1])
location = geolocator.reverse(Latitude+","+Longitude)
# object creation
obj = TimezoneFinder()
Local_Time_Zone = obj.timezone_at(lng=geocoder.ip('me').latlng[1], lat=geocoder.ip('me').latlng[0])
print("Time Zone : ", Local_Time_Zone)

## Getting the location of experiment

In [ ]:
file = open("QR_code_Result.txt",'w')
file.write("Time Zone = "+str(Local_Time_Zone)+"\n")
file.write("\n")
file.write("Location of experiment = "+str(location)+"\n")
file.write("\n")

## Import pandas for saving data to CSV file

In [ ]:
import pandas
time=datetime.now()
dod=str(time.month)+"_"+str(time.day)+"_"+str(time.year)
tod=str(time.hour)+"_"+str(time.minute)+"_"+str(time.second)+"_"+str(time.microsecond)
fname="Time_and_date_of_detection_"+dod+"_"+tod+".csv";
df = pandas.DataFrame(list())
df.to_csv(fname)

## The code below turns on a camera and extracts the QR code

In [ ]:
def main():
    #1
    frame_no=0;
    camera = cv2.VideoCapture(0) # cv2.VideoCapture(1) for external camera and cv2.VideoCapture(0) for laptop camera
    ret, frame = camera.read()
    QRlist=[]
    timelist=[]
    df2 = pandas.DataFrame(columns = ["frame number","Date of detection","Time of detection", "Location","Time Zone","QR Code reading","x","y","w","h"])
    #frame_no=camera.get(cv2.CAP_PROP_POS_FRAMES);

    #2
    while ret:
        frame_no+=1;
        ret, frame = camera.read()
        QRcodes = pyzbar.decode(frame)
        #Enter a snippet of code here to detect bags etc.
        for QRcode in QRcodes:
            x, y , w, h = QRcode.rect
            #1
            QRcode_info = QRcode.data.decode('utf-8')
            cv2.rectangle(frame, (x, y),(x+w, y+h), (255, 0, 0), 4)
            QRlist.append(QRcode_info)
            timelist.append(datetime.now())
            #2
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, QRcode_info, (x + 6, y - 6), font, 2.0, (255, 0, 0), 2)
            time=datetime.now()
            file.write("Recognized QR code = " + QRcode_info+"\n")
            file.write("Date of detection = " +str(time.month)+"/"+str(time.day)+"/"+str(time.year)+"\n")
            file.write("Time of detection = " +str(time.hour)+":"+str(time.minute)+":"+str(time.second)+":"+str(time.microsecond)+"\n")
            file.write("frame number = " + str(frame_no)+"\n")
            file.write("\n")
            dod=str(time.month)+"/"+str(time.day)+"/"+str(time.year)
            tod=str(time.hour)+":"+str(time.minute)+":"+str(time.second)+":"+str(time.microsecond)
            df2 = df2.append({"frame number":frame_no,"Date of detection":dod,"Time of detection":tod, "Location":str(location),"Time Zone":str(Local_Time_Zone),"QR Code reading":QRcode_info,"x":x,"y":y,"w":w,"h":h}, ignore_index = True)
            
        cv2.imshow('QR code reader', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
    
        
    camera.release()
    cv2.destroyAllWindows()
    df2.to_csv(fname)
#4
if __name__ == '__main__':
    main()
    